Объявляем системные переменные

In [ ]:
path_to_models = '/kaggle/input/gpt2got_v2'
path_to_books = '/kaggle/input/game-of-thrones-book-files'
output_prefix = 'got'

препроцессинг текстов и функция, которая возвращает последний снепшот модели из папки снепшотов

In [ ]:
import os
import glob
import re

def preprocessing(path_to_data):
    list_of_true_names = ['got5.txt', 'got1.txt', 'got2.txt', 'got4.txt', 'got3.txt']
    list_of_files = [os.path.join(path_to_data, i) for i in list_of_true_names]
    print(list_of_files)
    text = None
    for file in list_of_files:
        with open(file, 'r') as f:
            if text is None:
                text = f.read()
            else:
                text += f.read()
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.split('\n')
    text = [sentence.split('. ') for sentence in text]
    text = sum(text, [])
    text = [word for word in text if word != '' and word != '.']
#     return text

    new_list = []
    for i in range(0, (len(text) // 10) * 10, 10):
        new_list.append(text[i] + ' ' + 
                        text[i+1] + ' ' + 
                        text[i+2] + ' ' + 
                        text[i+3] + ' ' + 
                        text[i+4] + ' ' +
                        text[i+5] + ' ' + 
                        text[i+6] + ' ' + 
                        text[i+7] + ' ' + 
                        text[i+8] + ' ' + 
                        text[i+9])
    return new_list

def get_last_state_dict(path_to_data):
    list_of_files = glob.glob(f"{path_to_data}/*")
    array = []
    for file in list_of_files:
        array.append((file, int(re.findall('\d+', file.split('/')[-1])[0])))
    array.sort(key= lambda x: x[1], reverse=True)
    path, epoch = array[0]
    print(f'last state dict in {path}. epoch {epoch}')
    return path, epoch

In [ ]:
text = preprocessing(path_to_books)

In [ ]:
# https://githubhelp.com/EmilyNLP/Fine-Tune-GPT2-to-Generate-Stories

# http://education.abcom.com/using-gpt-2-to-write-like-shakespeare/

Датасеты и даталоадеры

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, PreTrainedTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

class GameOfThronesDataset2(Dataset):
    def __init__(self, list_of_sentences):
        self.labels=[]
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.data = self.tokenizer(list_of_sentences, padding=True, return_attention_mask=True)
        
        labels=[]
        for ids, attention_mask in zip(self.data['input_ids'],self.data['attention_mask']):
            label=ids.copy()
            real_len=sum(attention_mask)
            padding_len=len(attention_mask)-sum(attention_mask)
            label[:]=label[:real_len]+[-100]*padding_len
            labels.append(label)
            
        self.data['labels']=labels
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):

        return [torch.tensor(self.data['input_ids'][index], dtype=torch.long),
                torch.tensor(self.data['attention_mask'][index], dtype=torch.long),
                torch.tensor(self.data['labels'][index], dtype=torch.long)]

In [ ]:
train = text[:12000]
test = text[12000:]

In [ ]:
train_dataset = GameOfThronesDataset2(train)
test_dataset = GameOfThronesDataset2(test)

In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler

train_dataloader = DataLoader(train_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

инициализация

In [ ]:
from transformers import GPT2LMHeadModel
from transformers import AdamW
import numpy as np
# import gc
from transformers.optimization import get_linear_schedule_with_warmup

model = GPT2LMHeadModel.from_pretrained('gpt2')

num_train_epochs = 10
training_steps_per_epoch=len(train_dataloader)
total_num_training_steps = int(training_steps_per_epoch * num_train_epochs)
weight_decay = 0
learning_rate = 5e-5
adam_epsilon = 1e-8
warmup = 0.1
warmup_steps = int(total_num_training_steps * warmup)

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_num_training_steps
)

# gc.collect()
# torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# path, epoch = get_last_state_dict(path_to_models)

In [ ]:
# model.load_state_dict(torch.load(path))
# model.to(device)

Обучение

In [ ]:
from tqdm import tqdm

def train2(traindataloader, testdataloader, model, optimizer, scheduler, device, epochs, output_dir, output_prefix, save_model_on_epoch, validate_every):
    
    losses_train_overall = []
    losses_test_overall = []
    
    for i in range(epochs):
        
        print(f"Training epoch {i}")
        model.train()
        train_loss = []
        test_loss = []
        model.zero_grad() ###
        optimizer.zero_grad()
        for batch in tqdm(traindataloader, desc="train"):
            inputs, attention, labels = batch
            inputs = inputs.to(device)
#             inputs = inputs.squeeze(1).to(device)
            attention = attention.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=inputs, attention_mask=attention, labels=labels)
            loss = outputs[0]

            batch_loss = loss.cpu().item()
            train_loss.append(batch_loss)

            loss.backward()
            optimizer.step()
            scheduler.step()
            model.zero_grad() ###
            optimizer.zero_grad()
            del loss
            
        train_loss_mean = np.mean(train_loss)
        print(f'train loss = {train_loss_mean}')
        print()

        if (i % validate_every == 0):
            
            model.eval()
            optimizer.zero_grad()
            model.zero_grad()
            
            with torch.no_grad():
                               
                for batch in tqdm(testdataloader, desc="eval"):
                    inputs, attention, labels = batch
                    inputs = inputs.to(device)
#                     inputs = inputs.squeeze(1).to(device)
                    attention = attention.to(device)
                    labels = labels.to(device)
                    
                    outputs = model(input_ids=inputs, attention_mask=attention, labels=labels)
                    loss = outputs[0]  
                    
                    batch_val_loss = loss.cpu().item()
                    test_loss.append(batch_val_loss)
                    del loss
                    
                test_loss_mean = np.mean(test_loss)
                print(f'validate loss = {test_loss_mean}')
                print()

        if save_model_on_epoch:
            torch.save(model.state_dict(), os.path.join(output_dir, f"{output_prefix}-{epochs}.pt"))

        losses_train_overall.append(train_loss_mean)
        losses_test_overall.append(test_loss_mean)
    return losses_train_overall, losses_test_overall

list_train_loss, list_test_loss =  train2(
    train_dataloader, 
    test_dataloader, 
    model, 
    optimizer, 
    scheduler, 
    device, 
    epochs=num_train_epochs, 
    output_dir="/kaggle/working", 
    output_prefix='got_v2', 
    save_model_on_epoch=True, 
    validate_every=1
)

Кривые обучения

In [ ]:
import matplotlib.pyplot as plt

def learning_curves(train, val, txt='loss'):
    plt.figure(figsize=(8,6))
    plt.plot(range(1, len(train)+1), train, label='train {}'.format(txt))
    plt.plot(range(1, len(val)+1), val, label='validation {}'.format(txt))

    plt.title('Training history', fontsize=14)
    plt.ylabel('{}'.format(txt), fontsize=14)
    plt.xlabel('Epoch', fontsize=14)
    plt.legend(fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.grid()
    plt.show()

In [ ]:
learning_curves(list_train_loss, list_test_loss)

In [ ]:
from itertools import chain
learning_curves(list(a), list(b))

INFERENCE

In [2]:
from transformers import GPT2LMHeadModel
import torch

model = GPT2LMHeadModel.from_pretrained('gpt2')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load('/kaggle/input/model-snapshot/got_v2-20.pt'))

In [10]:
!pip install rpunct

Расставляем пунктуацию

In [11]:
from rpunct import RestorePuncts
# The default language is 'english'
rpunct = RestorePuncts()

Генерируем текст

In [17]:
from transformers import GPT2Tokenizer

dataset_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
dataset_tokenizer.pad_token = dataset_tokenizer.eos_token


def generate(text, size='small'):
    """
    size = small, middle, big
    """
    dict_of_size = {'small': 100, 'middle': 200, 'big': 300}
    ids = dataset_tokenizer.encode(text, return_tensors='pt').to(device)
    greedy_output = model.generate(ids, do_sample=True, max_length=dict_of_size[size], top_k=40, top_p=0.95, num_return_sequences=3)
    result = rpunct.punctuate(dataset_tokenizer.decode(greedy_output[0], skip_special_tokens=False))
    return result

generate('katelyn', 'small')